In [5]:
import nltk
import re
import numpy as np
import random
from collections import Counter
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- ВИПРАВЛЕННЯ: ЗАВАНТАЖЕННЯ ВСІХ НЕОБХІДНИХ РЕСУРСІВ ---
nltk.download('punkt')
nltk.download('punkt_tab')  # <--- ЦЕЙ РЯДОК ВИРІШУЄ ВАШУ ПОМИЛКУ

# --- ЗАВАНТАЖЕННЯ ДАНИХ ---
# Збережіть тексти Леся Подерв'янського у файл 'poderviansky.txt'
# і покладіть поруч із цим скриптом.

def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = f.read()
        return data
    except FileNotFoundError:
        print(f"Помилка: Файл {file_path} не знайдено. Будь ласка, завантажте файл.")
        return ""

def split_to_sentences(data):
    # Розбиваємо на речення, враховуючи специфіку (діалоги)
    # Можна просто за новим рядком, як у методичці
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    return sentences

def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        # Використовуємо NLTK для токенізації
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences

def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

# --- ПОПЕРЕДНЯ ОБРОБКА (ОБМЕЖЕННЯ СЛОВНИКА) ---

def count_words(tokenized_sentences):
    word_counts = Counter()
    for sentence in tokenized_sentences:
        word_counts.update(sentence)
    return word_counts

def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    word_counts = count_words(tokenized_sentences)
    # Залишаємо слова, що зустрічаються >= count_threshold разів
    closed_vocab = [word for word, count in word_counts.items() if count >= count_threshold]
    return closed_vocab

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocab_set = set(vocabulary)
    replaced_tokenized_sentences = []

    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocab_set:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)

    return replaced_tokenized_sentences

def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
def count_n_grams(data, n, start_token='<s>', end_token='</s>'):
    n_grams = Counter()

    for sentence in data:
        # Додаємо паддінг
        sentence = [start_token] * (n-1) + sentence + [end_token]
        sentence = tuple(sentence)

        # Слайдинг вікно
        for i in range(len(sentence) - n + 1):
            n_gram = sentence[i:i+n]
            n_grams[n_gram] += 1

    return dict(n_grams)

def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram)

    # Кількість попередньої n-грами (знаменник)
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size

    # Кількість n+1 грами (чисельник)
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)
    numerator = n_plus1_gram_count + k

    return numerator / denominator

def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    # Додаємо спецтокени до словника для перевірки всіх можливих варіантів
    vocab_plus = vocabulary + ['</s>', '<unk>']
    vocabulary_size = len(vocab_plus)

    probabilities = {}
    for word in vocab_plus:
        prob = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        probabilities[word] = prob

    return probabilities

In [9]:
def create_ui(train_data_processed, vocabulary):
    # Попередньо обчислюємо n-грами
    print("Обчислення N-грам (1-5)... Це може зайняти хвилину.")
    n_gram_counts_list = []
    for n in range(1, 6):
        # print(f"Обробка {n}-грам...") # Можна закоментувати, щоб не засмічувати вивід
        counts = count_n_grams(train_data_processed, n)
        n_gram_counts_list.append(counts)
    print("Готово! Інтерфейс завантажується...")

    # --- Віджети ---
    text_input = widgets.Text(
        value='', placeholder='Почніть писати...', description='Текст:',
        layout=widgets.Layout(width='600px')
    )

    prefix_input = widgets.Text(
        value='', placeholder='Фільтр (напр., "п")', description='Початок слова:',
        layout=widgets.Layout(width='300px')
    )

    n_gram_dropdown = widgets.Dropdown(
        options=[('Уніграми (1)', 1), ('Біграми (2)', 2), ('Триграми (3)', 3), ('4-грами', 4), ('5-грами', 5)],
        value=3, description='Модель:'
    )

    k_slider = widgets.FloatSlider(
        value=1.0, min=0.01, max=5.0, step=0.1, description='K (Smooth):',
        readout=True, readout_format='.1f'
    )

    suggestion_output = widgets.Output()

    # --- Логіка ---
    def on_suggestion_clicked(b):
        word = b.description.split(' ')[0]
        current = text_input.value
        if current and not current.endswith(' '):
            current += ' '
        text_input.value = current + word + ' '
        prefix_input.value = ''
        update_suggestions(None)

    def update_suggestions(_):
        with suggestion_output:
            clear_output()
            current_text = text_input.value.strip()
            tokens = nltk.word_tokenize(current_text.lower()) if current_text else []

            n = n_gram_dropdown.value
            k = k_slider.value

            needed_context = n - 1

            if n == 1:

                total_words = sum(n_gram_counts_list[0].values())
                probs = {w[0]: (c + k)/(total_words + k*len(vocabulary)) for w, c in n_gram_counts_list[0].items()}
            else:
                context = tokens[-needed_context:] if len(tokens) >= needed_context else ['<s>'] * (needed_context - len(tokens)) + tokens
                counts_context = n_gram_counts_list[n-2]
                counts_target = n_gram_counts_list[n-1]
                probs = estimate_probabilities(context, counts_context, counts_target, vocabulary, k)

            # Фільтрація
            start = prefix_input.value.lower()

            # Тепер w - це точно рядок, помилки не буде
            filtered = {w: p for w, p in probs.items() if w.startswith(start) and w not in ['<s>', '<unk>']}

            # Топ 5
            top_5 = sorted(filtered.items(), key=lambda x: x[1], reverse=True)[:5]

            if not top_5:
                print("Немає варіантів (спробуйте змінити N або K).")
                return

            buttons = []
            for w, p in top_5:
                btn = widgets.Button(description=f"{w} ({p:.3f})")
                btn.on_click(on_suggestion_clicked)
                buttons.append(btn)

            display(widgets.HBox(buttons))

    # Прив'язка подій
    text_input.observe(update_suggestions, names='value')
    prefix_input.observe(update_suggestions, names='value')
    n_gram_dropdown.observe(update_suggestions, names='value')

    # Вивід
    display(widgets.VBox([
        widgets.HBox([n_gram_dropdown, k_slider]),
        text_input,
        prefix_input,
        suggestion_output
    ]))

In [10]:
# Ім'я файлу з текстом (завантажте його заздалегідь!)
FILENAME = "poderviansky.txt"

# Якщо файлу немає, створимо демо-файл для перевірки коду
import os
if not os.path.exists(FILENAME):
    with open(FILENAME, "w", encoding="utf-8") as f:
        f.write("Гамлет, або Феномен датського кацапізму.\n")
        f.write("Купатися чи не купатися? Блядські ці питання зайбують.\n")
        f.write("Митець повинен бути голодним.\n")
        f.write("Лесь Подерв'янський — геній сучасності.\n")
    print("Створено демо-файл, оскільки оригінал не знайдено.")

# 1. Завантаження
raw_data = load_data(FILENAME)
print(f"Завантажено символів: {len(raw_data)}")

# 2. Токенізація
tokenized_docs = get_tokenized_data(raw_data)
random.shuffle(tokenized_docs)

# 3. Розділення на Train/Test
train_size = int(len(tokenized_docs) * 0.8)
train_docs = tokenized_docs[:train_size]
test_docs = tokenized_docs[train_size:]

# 4. Обробка (UNK)
train_data_proc, test_data_proc, vocab = preprocess_data(train_docs, test_docs, count_threshold=1)
print(f"Розмір словника: {len(vocab)}")

# 5. Запуск UI
create_ui(train_data_proc, vocab)

Завантажено символів: 166
Розмір словника: 19
Обчислення N-грам (1-5)... Це може зайняти хвилину.
Готово! Інтерфейс завантажується...
